In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
import numpy as np
import math
from typing import List, Optional, Tuple, Union


In [3]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)
    
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for num,hand in enumerate(results.multi_hand_landmarks):
        mp_drawing.draw_landmarks(
            image, hand, mp_hands.HAND_CONNECTIONS)
        if get_label(num,hand,results):
            text,coord=get_label(num,hand,results)
            cv2.putText(image,text,coord,cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2,cv2.LINE_AA)
            
            
            
    #hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  
    mask = cv2.inRange(image, (0, 255, 0), (0, 256,0))
    imask = mask>0
    green = np.zeros_like(image, np.uint8)
    green[imask] = image[imask]
    
    cv2.imshow('MediaPipe Hands', green)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
def get_label(index,hand,results):
    output=None
    for idx,classification in enumerate(results.multi_handedness):
        if classification.classification[0].index==index:
            label=classification.classification[0].label
            score=classification.classification[0].score
            text="{} {}".format(label,round(score,2))
            
            coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                                    [640,480]).astype(int))
            output=text,coords
    return output
            